In [97]:
from os import listdir
from os.path import isfile, join
from string import punctuation
from nltk.stem import PorterStemmer

In [98]:
ps = PorterStemmer()

# Stopwords

In [44]:
with open( './data/stopwords.txt', 'r' ) as f:
    stopwords = f.readlines()
stopwords = [ x.strip() for x in stopwords ] 

# Reading files

In [16]:
abstracts_path = './data/abstracts/'
abstracts_filenames = [ f for f in listdir( abstracts_path ) if isfile( join( abstracts_path, f ) ) ]

In [17]:
len( abstracts_filenames )

1347

In [18]:
gold_path = './data/gold/'
gold_filenames = [ f for f in listdir( gold_path ) if isfile( join( gold_path, f ) ) ]

In [19]:
len( gold_filenames )

1331

In [20]:
filenames =  list( set( abstracts_filenames ) & set( gold_filenames ) )

In [21]:
len( filenames )

1330

In [49]:
files = []
for filename in filenames:
    file = dict()
    file[ 'filename' ] = filename
    
    with open( abstracts_path + filename, 'r' ) as tmp_file:
        abstract = tmp_file.read().replace( '\n', ' ' )
    file[ 'abstract' ] = abstract
    
    with open( gold_path + filename, 'r' ) as tmp_file:
        gold = tmp_file.readlines()
    gold = [ w.replace('\n', '') for w in gold ]
    file[ 'gold' ] = gold
    
    files.append( file )

In [50]:
len( files )

1330

# Tokenization and removing punctuation

In [51]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [59]:
punctation2 = punctuation.replace( '\'', '' ).replace( '-', '' ).replace( '_', '' )
punctation2

'!"#$%&()*+,./:;<=>?@[\\]^`{|}~'

In [65]:
suffixes = ( '_nn', '_nns', '_nnp', '_nnps', '_jj' )

In [103]:
for file in files:
    words = ''.join( c.lower() for c in file[ 'abstract' ] if ( c not in punctation2 ) ).split()
    words = [ w.split( '_' )[ 0 ] for w in words if w.endswith( suffixes ) ]
    words = [ w for w in words if w not in stopwords ]
    try:
        words.remove( '' )
    except ValueError:
        pass
    words = [ ps.stem( w ) for w in words ]
    file[ 'tokenized_abstracts' ] = words

# Graph creation

In [129]:
for file in files:
    graph = dict()
    for i in range( len( file[ 'tokenized_abstracts' ] ) ):
        token = file[ 'tokenized_abstracts' ][ i ]
        if token not in graph:
            graph[ token ] = dict()
        try:
            next_word = file[ 'tokenized_abstracts' ][ i + 1 ]
            if next_word not in graph[ token ]:
                graph[ token ][ next_word ] = 1
            else:
                graph[ token ][ next_word ] = graph[ token ][ next_word ] + 1 
        except IndexError:
            pass
    file[ 'graph' ] = graph

In [131]:
files[ 1 ]

{'abstract': 'Online_JJ spelling_NN correction_NN for_IN query_NN completion_NN In_IN this_DT paper_NN ,_, we_PRP study_VBD the_DT problem_NN of_IN online_JJ spelling_NN correction_NN for_IN query_NN completions_NNS ._. Misspelling_NNP is_VBZ a_DT common_JJ phenomenon_NN among_IN search_NN engines_NNS queries_NNS ._. In_IN order_NN to_TO help_VB users_NNS effectively_RB express_VBP their_PRP$ information_NN needs_NNS ,_, mechanisms_NNS for_IN automatically_RB correcting_VBG misspelled_VBN queries_NNS are_VBP required_VBN ._. Online_JJ spelling_NN correction_NN aims_VBZ to_TO provide_VB spell_NN corrected_JJ completion_NN suggestions_NNS as_IN a_DT query_NN is_VBZ incrementally_RB entered_VBN ._. As_IN latency_NN is_VBZ crucial_JJ to_TO the_DT utility_NN of_IN the_DT suggestions_NNS ,_, such_PDT an_DT algorithm_NN needs_VBZ to_TO be_VB not_RB only_RB accurate_JJ ,_, but_CC also_RB efficient_JJ ._. To_TO tackle_VB this_DT problem_NN ,_, we_PRP propose_VBP and_CC study_VBP a_DT generative